In [169]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
import json

In [170]:
def load_cookies(driver: webdriver.Firefox, social_media: str, file: str) -> bool:
    """Charger et injecter les cookies Facebook"""
    try:
        driver.get(social_media)
        time.sleep(3)
        
        with open(file, "r", encoding="utf-8") as file:
            cookies_data = json.load(file)
        
        # Extraire la liste des cookies du dictionnaire
        cookies = cookies_data["cookies"] if "cookies" in cookies_data else cookies_data
        
        print(f"{len(cookies)} cookies chargés depuis le fichier")
        
        for cookie in cookies:
            # Créer une copie du cookie pour éviter de modifier l'original
            clean_cookie = cookie.copy()
            
            # Supprimer les attributs qui peuvent causer des problèmes avec Selenium
            for key in ["sameSite", "storeId", "id", "hostOnly", "session"]:
                clean_cookie.pop(key, None)
                
            try:
                driver.add_cookie(clean_cookie)
            except Exception as e:
                print(f"Erreur avec le cookie {cookie.get('name', 'inconnu')}: {e}")
        
        print("Cookies injectés avec succès")
        return True
        
    except FileNotFoundError:
        print("Fichier facebook_cookies.json non trouvé!")
        return False
    except Exception as e:
        print(f"Erreur lors du chargement des cookies: {e}")
        return False

In [171]:
def init_driver() -> webdriver.Firefox:
  opts = Options()
  
  # avec firefox

  # Forcer un user-agent desktop
  # desktop_ua = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36"
  # opts.set_preference("general.useragent.override", desktop_ua)
  # Ajuster l'échelle pour éviter rendu mobile
  # opts.set_preference("layout.css.devPixelsPerPx", "1.0")

  # srv = Service(executable_path=r'/snap/bin/firefox.geckodriver')
  # driver = webdriver.Firefox(options=opts, service=srv)
  # driver.set_window_size(1920, 1080)
  # Ou avec chrome 

  
  desktop_ua = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0  0.0 Safari/537.36"
  opts.add_argument(f"--user-agent={desktop_ua}")
  opts.add_argument("--window-size=1920,1080")
  opts.add_argument("--disable-blink-features=AutomationControlled")

  driver = webdriver.Chrome(options=opts)
  
  return driver


In [172]:
def extract_comments_text(post_container, driver) -> list:
    """Extraire le texte des commentaires depuis le conteneur de post"""

    try:
        show_comments_btn = post_container.find_element(By.XPATH, './/div/div/div/div/div/div/div/div/div/div/div/div[13]/div/div/div[4]/div/div/div[1]/div/div[1]/div/div[2]/div[2]')
        show_comments_btn.click()
        time.sleep(10)
        # comment_dialog = driver.find_element(By.XPATH, '//div[contains(@role, "dialog")]')

        close_dialog_btn = driver.find_element(By.XPATH, '//div[contains(@role, "dialog")]//div[contains(@aria-label, "Fermer")]')

        scrollable_container = driver.find_element(By.XPATH, '//div[contains(@role, "dialog")]/div/div[1]/div/div[2]')

        # Essayer de trouver le conteneur de commentaires, sinon utiliser le conteneur scrollable
        try:
            comments_container = scrollable_container.find_element(By.XPATH, './/div[2]/div/div/div[2]/div/div[2]')
            print("Conteneur de commentaires trouvé")
        except NoSuchElementException:
            print("Conteneur de commentaires non trouvé - utilisation du conteneur scrollable")
            print("fermeture du dialogue des commentaires")
            close_dialog_btn.click()
            
            return []
        
        comments = []
        previous_count = 0
        no_change_iterations = 0
        max_scrolls = 30  # Sécurité
        scroll_count = 0
        
        while scroll_count < max_scrolls:

            # try:

            #     voir_reponses_btns = comments_container.find_elements(
            #         By.XPATH, 
            #         ".//div[contains(text(), 'Voir') and (contains(text(), 'réponse') or contains(text(), 'réponses'))]"
            #     )
            #     for btn in voir_reponses_btns:
            #         try:
            #             btn.click()
            #             time.sleep(2)
            #         except:
            #             continue
            # except NoSuchElementException:
            #     print("some shit")
            #     pass
                
            comments_text = comments_container.find_elements(By.XPATH, './/div/span[@dir="auto"]')
            current_count = len(comments_text)
            
            # print(current_count, previous_count)
            # Si le nombre n'a pas changé après plusieurs tentatives
            if current_count == previous_count:
                no_change_iterations += 1
                if no_change_iterations >= 6:  # 3 fois sans changement = fin
                    print("Fin du chargement des commentaires")
                    break
            else:
                no_change_iterations = 0  # Reset si nouveaux commentaires
            
            previous_count = current_count
            scroll_count += 1
            
            # Scroller
            driver.execute_script("""
                var container = arguments[0];
                container.scrollBy(0, container.clientHeight);
            """, scrollable_container)
            
            time.sleep(6)  
        

        comments = [comment.text for comment in comments_text if comment.text.strip() != '']
        print("fermeture du dialogue des commentaires")
        close_dialog_btn.click()
        time.sleep(2)
        
    except Exception as e:
        print(f"Erreur': {e}")
        return []

    return set(comments)

In [173]:
def scraped_facebook_comments(driver: webdriver.Firefox, group_link) -> list:
    """Scraper les commentaires d'un post Facebook donné"""
    driver.get(group_link)
    time.sleep(15)
    
    comments = []
    previous_count = 0
    no_change_iterations = 0
    all_ready_taken = set()
    max_scrolls = 300 
    scroll_count = 0

    # container = driver.find_element(By.CSS_SELECTOR, 'div[id*="mount"]')
    post_container_height = 0

    while scroll_count < max_scrolls:
        div_ctn_index = len(driver.find_elements(By.XPATH, '//*[contains(@id, "mount")]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div[4]/div[2]/div/div[2]/div'))
        
        post_containers = driver.find_elements(By.XPATH, '//*[contains(@id, "mount")]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div[4]/div[2]/div/div[2]/div[last()]/div')

        if post_container_height == 0 and len(post_containers) > 0:
            post_container_height = driver.execute_script("""
                return arguments[0].clientHeight;
            """, post_containers[0])

        for post in post_containers:
            try:
                post_link = post.find_elements(By.XPATH, ".//a")[2].get_attribute('href')
                print(f"Traitement du post: {post_link}")
                if post_link.lower() in all_ready_taken:
                    continue
                
                all_ready_taken.add(post_link.lower())
                
                new_comments = extract_comments_text(post, driver)
                comments.extend(list(new_comments))
                print(f"  → {len(new_comments)} commentaires récupérés")
                
            except Exception as e:
                print(f"Erreur sur un post: {e}")
                continue
        
        current_count = len(all_ready_taken)
        print(f"Total posts facebook traités: {current_count}")
        print(f"Total commentaires facebook: {len(comments)}")
        
        # Vérifier si on a de nouveaux posts
        print(no_change_iterations)
        if current_count == previous_count and current_count != 0:
            no_change_iterations += 1
            if no_change_iterations >= 50:
                print("no change in posts, stopping scroll three times")
                break
        else:
            no_change_iterations = 0
        
        previous_count = current_count
        scroll_count += 1
        
        # Scroller de 5 posts (5 × hauteur d'un post)
        if post_container_height > 0:
            scroll_amount = post_container_height * 5
            driver.execute_script("""
                var scrollAmount = arguments[0];
                window.scrollBy({
                    top: scrollAmount,
                    behavior: 'smooth'
                });
            """, scroll_amount)
        else:
            # Fallback si hauteur pas détectée
            driver.execute_script("window.scrollBy(0, 3000);")
        
        print("Scrolling...", scroll_count, max_scrolls )
        time.sleep(3)  

    
    print(f"\n=== Scraping terminé ===")
    print(f"Total posts traités: {len(all_ready_taken)}")
    print(f"Total commentaires: {len(comments)}")

    
    return comments

In [174]:
def scroll_at_bottom(driver: webdriver.Firefox, pause_time: int = 2):
    """Faire défiler la page jusqu'en bas pour charger plus de contenu"""
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def scraped_linkedin_comments(driver: webdriver.Firefox) -> list:
    """Scraper les commentaires d'un post LinkedIn donné"""
    driver.get("https://www.linkedin.com/company/fnac/")
    time.sleep(5)
    
    driver.find_element(By.CSS_SELECTOR, 'a.fastrack-sign-in-cta').click()
    comments = []
    
    scroll_at_bottom(driver, pause_time=5)
    posts = driver.find_elements(By.CSS_SELECTOR, "div.fie-impression-container")
    for post in posts: 
        try:
            show_comments_btn = post.find_element(By.CSS_SELECTOR, "ul ul > li > button[aria-label*='comm']")
            show_comments_btn.click()
            time.sleep(5)
            
            while True:
                try:
                    post.find_element(By.CSS_SELECTOR, "a[data-test-id='main-feed-activity-card-with-comments__see-more-comments']").click()
                except NoSuchElementException:
                    break
                time.sleep(3)

            comments = post.find_elements(By.CSS_SELECTOR, "span.comments-comment-item__main-content")
            for comment in comments:
                comments.append(comment.text)
        except NoSuchElementException :
            continue

    return comments

In [175]:
social_media = [
  # {
  #   "name": "facebook",
  #   "url": "https://web.facebook.com",
  #   "page_url": "https://web.facebook.com/Fnac",
  #   "scrape_function": scraped_facebook_comments,
  #   "cookies_file": "facebook.json"
  # },
  {
    "name": "facebook",
    "url": "https://web.facebook.com",
    "page_url": "https://www.facebook.com/wavecotedivoire",
    "scrape_function": scraped_facebook_comments,
    "cookies_file": "facebook.json"
  },
]

In [176]:
comments = list()
for sm in social_media:
    driver = init_driver()
    if load_cookies(driver, sm['url'], sm['cookies_file']):
        driver.refresh()
    time.sleep(15)

    comments.extend(sm["scrape_function"](driver, sm['page_url']))
    
    driver.quit()


df = pd.DataFrame({
    'comments': comments
})
df.to_csv('comments_wave.csv')

df.head()

10 cookies chargés depuis le fichier
Cookies injectés avec succès
Traitement du post: https://web.facebook.com/wavecotedivoire?__cft__[0]=AZXUW4hkp_u37EIJMqkbInCzonNwJna7KFd4WNsuv3BCSS1AeKGr-lidIaKu7X6CSr5lCV8mXCQb6NVDkanCx6kAaPuF0_iH4bRds0A6eN1sFSXPdOgS03XkICP9GKaUn2mpffviQcjL0giGcY6U5nXYkGRTXYD6LkQ4L0Q3wEyxS1KJSFUSr003bdm1-Wk2xhA&__tn__=%2CO%2CP-R#?gdi
Conteneur de commentaires trouvé


KeyboardInterrupt: 

In [ ]:
df.comments.nunique(), len(df)

(1427, 1477)

In [ ]:
pd.DataFrame({
  'comments': df.comments.unique()
}).to_csv('comments_cleaned_wave.csv')

In [ ]:
# !uv pip install python-docx wordcloud spacy fasttext --quiet

In [ ]:
# !python3 -m spacy download fr_core_news_sm
# !python3 -m spacy download en_core_web_sm

In [ ]:
# !uv pip install fasttext